# Worksheet 07

Name:  Tianyi Bao
UID: U89641075

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[5.310979740095483, 6.597493215171307, 5.463725605679677, 4.338444936876503, 3.4185419986565946, 4.174862337597262, 6.118435247944875, 4.3119045875574065, 6.1162087132004705, 5.479291923284024]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
c2 = np.random.normal(8, 1, 10).tolist()
print(c2)

[8.422969201242056, 7.539026959411371, 9.090407678298297, 8.822124084362363, 7.144341114858729, 7.09236370095041, 7.96382912600709, 8.53950221559923, 5.710003725483641, 5.386750050545731]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[5.479291923284024, 5.386750050545731, 6.1162087132004705, 4.3119045875574065, 5.710003725483641, 6.118435247944875, 4.174862337597262, 8.53950221559923, 7.96382912600709, 7.09236370095041]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

1. mean
2. variance
3. P(Cj) 

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) ,len(c2) /(len(c1)+len(c2))]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

/home/codespace/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[5.479291923284024, 5.386750050545731, 6.1162087132004705, 4.3119045875574065, 5.710003725483641, 6.118435247944875, 4.174862337597262]
[8.53950221559923, 7.96382912600709, 7.09236370095041]
P(C_1) = 0.7,  P(C_2) = 0.3
mean_1 = 5.328208083659058,  mean_2 = 7.865231680852244
var_1 = 0.5400720814398821,  var_2 = 0.3538957081921967


The means for c1 and c2 are close but the variance is far from the true values. 

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [5]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0]*prob_c[0] /prob_x)
    prob_c1_x.append( pdf_i[1]*prob_c[1]/prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  5.479291923284024
probability of observing that point if it came from cluster 0 =  0.7103373976533739
probability of observing that point if it came from cluster 1 =  1.5202301274022228e-10
point =  5.386750050545731
probability of observing that point if it came from cluster 0 =  0.7343564229635331
probability of observing that point if it came from cluster 1 =  2.5200480069446346e-11
point =  6.1162087132004705
probability of observing that point if it came from cluster 0 =  0.2547883258100726
probability of observing that point if it came from cluster 1 =  5.599404366192115e-06
point =  4.3119045875574065
probability of observing that point if it came from cluster 0 =  0.1257503055333961
probability of observing that point if it came from cluster 1 =  1.4474595542324495e-22
point =  5.710003725483641
probability of observing that point if it came from cluster 0 =  0.5753572454833955
probability of observing that point if it came from cluster 1 =  9.963985137704596e-09
point

The estimated probabilities does seem correct. 

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [6]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]
var = [ sum(map(lambda x: x[0]*(x[1] - mean[0])**2, zip(prob_c0_x, data))) ,  sum(map(lambda x: x[0]*(x[1] - mean[1])**2, zip(prob_c1_x, data)))]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.707406119748872,  P(C_2) = 0.29259388025112787
mean_1 = 5.346681746246257,  mean_2 = 7.884784784947439
var_1 = 4.00867724151313,  var_2 = 1.0163833935269675


The means are similar, but the varience is far different from the ones produced from K means. 

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [8]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0]*prob_c[0] /prob_x)
    prob_c1_x.append( pdf_i[1]*prob_c[1]/prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))

prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]
var = [ sum(map(lambda x: x[0]*(x[1] - mean[0])**2, zip(prob_c0_x, data))) ,  sum(map(lambda x: x[0]*(x[1] - mean[1])**2, zip(prob_c1_x, data)))]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

point =  5.479291923284024
probability of observing that point if it came from cluster 0 =  0.03988398172468397
probability of observing that point if it came from cluster 1 =  0.11012387021975929
point =  5.386750050545731
probability of observing that point if it came from cluster 0 =  0.03987625893988555
probability of observing that point if it came from cluster 1 =  0.10810808877604801
point =  6.1162087132004705
probability of observing that point if it came from cluster 0 =  0.03984452983486243
probability of observing that point if it came from cluster 1 =  0.12195227656183504
point =  4.3119045875574065
probability of observing that point if it came from cluster 0 =  0.03953789971401451
probability of observing that point if it came from cluster 1 =  0.08150822911088179
point =  5.710003725483641
probability of observing that point if it came from cluster 0 =  0.039888368405237865
probability of observing that point if it came from cluster 1 =  0.11485037010541055
point =  6.1

Variance becomes larger while means are still similar. 

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [9]:
label = []
for p in probs:
  if p[1] > p[2]:
    label.append(0)
  else:
    label.append(1)


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      1       |
| A  C |      0       |
| A  D |      0       |
| A  E |      0       |
| B  C |      1       |
| B  D |      0       |
| B  E |      0       |
| C  D |      0       |
| C  E |      0       |
| D  E |      1       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

N!/(2!(N-2)!)

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

9!/(2!7!) = 36

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

For cluster 1, there is 1 pair that agrees with C. For cluster 2, there are 6 pairs that agree with C. There are 36-10 = 26 pairs of disagreements between C and P.

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.

Group the same cluster of C and P using filtering and calculate the agreements for each cluster. Use the maximum number of disagreements to minus the total number of agreements at the end. 